# Add a counties field to the stations file and map the cities to their appropriate county
Unfortunately, the original data source for the cities -> counties is incomplete, requiring a fair bit of manual entry to complete the table.

In [ ]:
import pandas as pd

### Station data

In [ ]:
# Read in the data, filter to just CA, and remove unnecessary columns
stations=pd.read_csv('Data/Weather/raw_with_headers/NOAA_ghcnd_stations.csv')
stations=stations.query('STATE == "CA"')
stations.head()

In [ ]:
## strip the trailing numbers and cardinal directions from city names
import re
for i, row in stations['NAME'].iteritems():
    new_row= re.sub(r'\s\d+\.*\d*.+', '', row) # number(s) . number(s) letter(s)
    new_row2 = re.sub(r'Ph|Rs|Nm|Fire\sDept|Hosp|Pk|Sp|Stn|Dam|Cdf|Rch|Hd|Gs|Rng|Ap|vr|Plt|Blvd|Fd|', '', new_row) # remove 2-3 letter acronyms
    stations.at[i, 'NAME']=new_row2.title() ## change the name to title case and replace the original value
stations['NAME'] = stations['NAME'].str.strip()
stations['NAME'] = stations['NAME'].str.replace(r'\s+', ' ')
stations.head()

In [ ]:
for i, row in stations['NAME'].iteritems():
    new_row= re.sub(r'\sCalifornia', '', row) # Remove California from the row (they're all in CA anyway)
    stations.at[i, 'NAME']=new_row

### City & County data

In [ ]:
## explicitly naming the columns to match up for the merge later on.
counties=pd.read_csv('Data/Counties/cal_populations_city.csv', header=0, names=['COUNTY', 'NAME'], usecols=[0,1])
counties['NAME'] = counties['NAME'].str.strip()
counties['COUNTY'] = counties['COUNTY'].str.strip()
counties['NAME'] = counties['NAME'].str.replace(r'\s+', ' ')
counties['COUNTY'] = counties['COUNTY'].str.replace(r'\s+', ' ')
counties.to_csv('Data/Counties/California_counties_kaggle.csv')
counties.head()

In [ ]:
#New file
counties2=pd.read_csv('Data/Counties/county_extract.csv', header=0, names=['COUNTY', 'NAME'], usecols=[0,1])
counties2['NAME'] = counties2['NAME'].str.strip()
counties2['COUNTY'] = counties2['COUNTY'].str.strip()
counties2['NAME'] = counties2['NAME'].str.replace(r'\s+', ' ')
counties2['COUNTY'] = counties2['COUNTY'].str.replace(r'\s+', ' ')
counties2.to_csv('Data/Counties/California_counties_google_sheet.csv')
#counties2.head()

In [ ]:
counties3=pd.concat([counties,counties2]).reset_index(drop=True)
#counties3.to_csv('Data/with_headers/California_counties_combined.csv')
counties3.head()

### Merge the two

In [ ]:
## join the stations and counties dataframes
stations=stations.merge(counties3, how='left', on='NAME')
stations.drop_duplicates(subset=None, keep='first', inplace=True)
#stations.to_csv('Data/with_headers/NOAA_ghcnd_stations_CA.csv')
stations.head()

### Optionally remove rows that don't have a county. These are mostly specific locations rather than cities

In [ ]:
import numpy as np
print(len(stations))
stations['COUNTY'].replace('', np.nan, inplace=True)
stations.dropna(subset=['COUNTY'], inplace=True)
print(len(stations))
stations.to_csv('Data/Weather/raw_with_headers/NOAA_ghcnd_stations_CA_blanks_removed.csv')
